In [1]:
import os
from timeit import default_timer as timer
from glob import glob
import pandas as pd
import numpy as np
import re

In [2]:
country_code = "US"
if os.getenv('CLUSTER')=='PRINCE':
    path_to_data='/scratch/spf248/twitter/data'
else:
    path_to_data='../../data'

# Geo id data

In [3]:
user_location_2_geo_id=pd.read_csv(glob(os.path.join(path_to_data,'official','city',country_code,'*user_id.csv'))[0],index_col=0).drop('index',1,errors='ignore').rename(columns={'NAME':'metro_area_name','metro_area':'metro_area_name'})

In [4]:
if country_code=='US':
    # Create Geo Id of Metro Areas
    user_location_2_geo_id.rename(columns={'geo_id':'geo_admin'},inplace=True)
    metro_area_name_2_geo_id=user_location_2_geo_id.drop_duplicates(
    'metro_area_name').reset_index(drop=True).reset_index(
    ).set_index('metro_area_name')['index'].rename('geo_id')
    user_location_2_geo_id['geo_id']=user_location_2_geo_id['metro_area_name'].apply(
    lambda x:metro_area_name_2_geo_id[x])

In [5]:
user_location_2_geo_id.to_csv(os.path.join(path_to_data,'official','city',country_code,'user_location_2_geo_id.csv'))

In [6]:
user_location_2_geo_id.head()

,user_location,geo_admin,metro_area_name,geo_id
0,New York,CN3606100000000,New York,0
1,"New York, NY",CN3606100000000,New York,0
2,NYC,CN3606100000000,New York,0
3,New York City,CN3606100000000,New York,0
4,NY,CN3606100000000,New York,0


In [7]:
user_location_2_geo_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6244 entries, 0 to 6243
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_location    6244 non-null   object
 1   geo_admin        6244 non-null   object
 2   metro_area_name  6244 non-null   object
 3   geo_id           6244 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 243.9+ KB


# Population data

In [8]:
pop_metro_areas=pd.read_csv(glob(os.path.join(path_to_data,'official','city',country_code,'*metro_areas_population*.csv'))[0],index_col='geo_id')
if country_code=="US":
    # Aggregate from admin 2 to metro areas
    pop_metro_areas=pop_metro_areas.reset_index().rename(columns={'geo_id':'geo_admin'}).merge(user_location_2_geo_id,on='geo_admin').set_index('geo_id').filter(regex='pop').groupby('geo_id').sum()
pop_metro_areas.drop([x for x in pop_metro_areas.columns if 'pop' not in x],1,inplace=True)
pop_metro_areas.columns.name='year'
pop_metro_areas=pop_metro_areas.rename(columns=lambda x:int(re.findall('(\d+)',x)[0])).T.stack().sort_index().rename('pop').reset_index()
pop_metro_areas.to_csv(os.path.join(path_to_data,'official','city',country_code,'pop_metro_areas.csv'))

In [9]:
pop_metro_areas.head()

,year,geo_id,pop
0,2007,0,6.103865e+08
1,2007,1,3.919142e+09
2,2007,2,7.310571e+08
3,2007,3,4.864556e+07
4,2007,4,6.552245e+07


In [10]:
pop_metro_areas.describe()

,year,geo_id,pop
count,4424.000000,4424.000000,4.424000e+03
mean,2013.500000,157.500000,3.045548e+07
std,4.031585,91.231197,2.358359e+08
min,2007.000000,0.000000,7.900888e+03
25%,2010.000000,78.750000,4.371324e+05
50%,2013.500000,157.500000,1.418567e+06
75%,2017.000000,236.250000,5.060050e+06
max,2020.000000,315.000000,4.121513e+09


In [11]:
pop_metro_areas.drop_duplicates(['year','geo_id']).shape[0]/pop_metro_areas.shape[0]

1.0

In [12]:
pop_metro_areas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    4424 non-null   int64  
 1   geo_id  4424 non-null   int64  
 2   pop     4424 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 103.8 KB


# Labor market data

In [13]:
if country_code!='US':
    labor_market_stats=pd.read_csv(os.path.join(path_to_data,'official','city',country_code,'labor_market_stats.csv'),sep={'AR':';','BR':',','CO':';','MX':','}[country_code])
    labor_market_stats.rename(columns={'value':'unemployment_rate','NAME':'metro_area_name'},inplace=True)
    labor_market_stats['date']=pd.to_datetime(labor_market_stats.apply(lambda x:str(x['year'])+'-Q'+str(x['quarter']),1)) + pd.offsets.QuarterEnd(0)
    labor_market_stats['month'] = labor_market_stats['date'].apply(lambda x:x.month)
    labor_market_stats = labor_market_stats[['year','month','geo_id','unemployment_rate']].sort_values(
    by=['year','month','geo_id']).reset_index(drop=True)
    if country_code =='CO':
        labor_market_stats.unemployment_rate=labor_market_stats.unemployment_rate.apply(lambda x:np.float(x.replace(',','.'))/100)
    labor_market_stats.to_csv(os.path.join(path_to_data,'official','city',country_code,'time_series_unemployment_rate.csv'))
else:
#     compute average rate across counties
    labor_market_stats=pd.read_csv(os.path.join(path_to_data,'official','city',country_code,'labor_market_stats.csv'),index_col=0).rename(columns={'geo_id':'geo_admin'})
    pop_admins=pd.read_csv(glob(os.path.join(path_to_data,'official','city',country_code,'*metro_areas_population*.csv'))[0]).rename(columns={'geo_id':'geo_admin','NAME':'metro_area_name'}).drop('UC_NM_LST',1,errors='ignore').sort_values(by=['metro_area_name','geo_admin']).set_index(['metro_area_name','geo_admin'])
    pop_admins.drop([x for x in pop_admins.columns if 'pop' not in x],1,inplace=True)
    pop_admins.columns.name='year'
    pop_admins=pop_admins.rename(columns=lambda x:int(re.findall('(\d+)',x)[0])).stack().sort_index().rename('pop').reset_index()
    labor_market_stats=labor_market_stats.merge(pop_admins,on=['year','geo_admin']).drop('geo_admin',1)
    labor_market_stats['unemployment_rate']=labor_market_stats['unemployment_rate'].multiply(labor_market_stats['pop'])
    labor_market_stats=labor_market_stats.groupby(['year','month','metro_area_name']).sum()
    labor_market_stats=labor_market_stats['unemployment_rate'].divide(labor_market_stats['pop']).rename('unemployment_rate')
    labor_market_stats/=100
    labor_market_stats=labor_market_stats.to_frame().reset_index()
    labor_market_stats['geo_id']=labor_market_stats['metro_area_name'].apply(lambda x:metro_area_name_2_geo_id.get(x,np.nan))
    labor_market_stats=labor_market_stats.sort_values(by=['year','month','geo_id']).reset_index(drop=True).drop(['metro_area_name'],1).dropna()
    labor_market_stats['geo_id']=labor_market_stats['geo_id'].astype(int)
    labor_market_stats=labor_market_stats[['year', 'month', 'geo_id', 'unemployment_rate']].copy()
    labor_market_stats.to_csv(os.path.join(path_to_data,'official','city',country_code,'time_series_unemployment_rate.csv'))

/Users/samuelfraiberger/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
labor_market_stats.head()

,year,month,geo_id,unemployment_rate
0,2007,1,0,0.049351
1,2007,1,1,0.049097
2,2007,1,2,0.052527
3,2007,1,3,0.045802
4,2007,1,4,0.034297


In [15]:
labor_market_stats.describe()

,year,month,geo_id,unemployment_rate
count,49916.000000,49916.000000,49916.000000,49916.000000
mean,2013.174453,6.400232,157.703181,0.064470
std,3.852548,3.469944,91.472435,0.031079
min,2007.000000,1.000000,0.000000,0.011000
25%,2010.000000,3.000000,78.000000,0.041813
50%,2013.000000,6.000000,157.000000,0.057000
75%,2016.000000,9.000000,238.000000,0.080781
max,2020.000000,12.000000,315.000000,0.335000


In [16]:
labor_market_stats.drop_duplicates(['year','month','geo_id']).shape[0]/labor_market_stats.shape[0]

1.0

In [17]:
labor_market_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49916 entries, 0 to 52935
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               49916 non-null  int64  
 1   month              49916 non-null  int64  
 2   geo_id             49916 non-null  int64  
 3   unemployment_rate  49916 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 1.9 MB
